# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import and configure gmaps API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the mouse data and the study results
city_data_df = pd.read_csv("../WeatherPy/output_data/city_data.csv")
len(city_data_df)
city_data_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind speed (mph),Cloud coverage,Datetime
0,Vaini,TO,-21.20,-175.20,77.00,94,9.17,75,1592781327
1,Bonavista,CA,48.65,-53.11,53.42,70,18.41,97,1592781459
2,Salalah,OM,17.02,54.09,82.40,88,5.82,40,1592781182
3,Butaritari,KI,3.07,172.79,83.68,79,14.47,56,1592781057
4,Hobart,AU,-42.88,147.33,53.60,93,8.05,75,1592781334


### Humidity Heatmap
* Configure gmaps.
* Use the Latitude and Longitude as locations and Humidity as the weight.
* Add Heatmap layer to map.
* For max intensity in the heat map, try setting it to the highest humidity found in the data set.

In [3]:
# Configure gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
# !pip install gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Store latitude and longitude in locations
locations = city_data_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_data_df["Humidity (%)"].astype(float)

# For max intensity in the heat map, try setting it to the highest humidity found in the data set.
highest_humidity = city_data_df["Humidity (%)"].max()

# Create heat layer using the Latitude and Longitude as locations and Humidity as the weight
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = highest_humidity, point_radius = 1.9, opacity = 1)

# Customize the size of the figure
figure_layout = {
    "width": "910px",
    "height": "740px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

# Plot the Heatmap
fig = gmaps.figure(layout = figure_layout, map_type = "HYBRID", zoom_level = 1.9, center = [24, 19])

# Add the heat layer to the map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='740px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.
* Drop any rows will null values and those that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
# Using vectorization operation, filter out all those rows which satisfy the given condition
# Using the drop function, narrow the City data DataFrame and create a new DataFrame that contains the ideal weather conditions
city_data_df.drop(city_data_df[city_data_df["Temperature (F)"] >= 83].index, inplace = True)
city_data_df.drop(city_data_df[city_data_df["Temperature (F)"] <= 74].index, inplace = True) 
city_data_df.drop(city_data_df[city_data_df["Wind speed (mph)"] >= 11].index, inplace = True) 
city_data_df.drop(city_data_df[city_data_df["Cloud coverage"] != 0].index, inplace = True) 
len(city_data_df)

7

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
* Store the first Hotel result into the DataFrame.
* Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country

In [6]:
# Create a new DataFrame that contains the ideal weather conditions
hotel_df = city_data_df

# Add "Hotel Name" column to the DataFrame
# Note that we used "" to specify initial entry
hotel_df["Hotel Name"] = ""

# Display the DataFrame with the new column
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind speed (mph),Cloud coverage,Datetime,Hotel Name
39,Sakakah,SA,29.97,40.21,77.00,23,4.70,0,1592781406,
67,Atuona,PF,-9.80,-139.03,80.20,66,9.55,0,1592781310,
161,Touros,BR,-5.20,-35.46,75.20,88,3.36,0,1592781312,
357,Tamandaré,BR,-8.76,-35.10,76.03,75,9.51,0,1592781506,
389,Archidona,ES,37.10,-4.39,75.99,41,2.82,0,1592781509,


In [7]:
# Use the search terms "Hotel" and our latitude and longitude
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# Create a params dictionary that will be updated with new hotel name for each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the latitude and longitude we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # Change the location with each iteration while leaving original params in place
    params["location"] = f"{latitude},{longitude}"

    # Make request and convert to json
    hotel_name = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f" Skipping since a hotel wasn't found...")

In [8]:
# Use the DataFrame replace and drop functions to omit rows without a hotel name
hotel_df.replace("", np.nan, inplace = True)
hotel_df.dropna(inplace = True)

# Reset the index
hotel_df.reset_index(drop = True, inplace = True)

In [9]:
# Create a new DataFrame that contains the final list of cities with corresponding hotel names
narrowed_city_df = hotel_df[["City", "Country", "Latitude", "Longitude", "Hotel Name"]]
narrowed_city_df

,City,Country,Latitude,Longitude,Hotel Name
0,Sakakah,SA,29.97,40.21,Fakhamat Al Orjoana Apartment
1,Atuona,PF,-9.80,-139.03,Villa Enata
2,Touros,BR,-5.20,-35.46,INN NEW HORIZON
3,Tamandaré,BR,-8.76,-35.10,Pousada 100 Passos
4,Archidona,ES,37.10,-4.39,Hotel Escua
5,Hīt,IQ,33.64,42.83,بيت اسماعبل محمد حمد
6,Piaçabuçu,BR,-10.41,-36.43,Piaçabuçu News


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotels = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Store latitude and longitude in locations
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity (%)"]

# Create heat layer using the Latitude and Longitude as locations and Humidity as the weight
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = highest_humidity, point_radius = 1.9, opacity = 1)

# Plot Heatmap
fig = gmaps.figure(layout = figure_layout, map_type = "HYBRID", zoom_level = 2, center = [24, 19])

# Add the heat layer
fig.add_layer(heat_layer)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotels, info_box_content = hotel_info)

# Add the markers and heat layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Customize the size of the figure
figure_layout = {
    "width": "910px",
    "height": "740px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='740px', margin='0 auto 0 auto', padding='1px', wi…